In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from tqdm import tqdm
import random

In [3]:
import malaya

In [4]:
# model = malaya.dependency.transformer(model = 'xlnet', quantized = True)

In [5]:
penguat_list = [
    'paling',
    'agak',
    'sungguh',
    'amat',
    'terlalu',
    'nian',
    'benar',
    'paling',
    'sangat'
]
end_penguat_list = ['sekali', 'sungguh', 'sangat']

In [6]:
set_penguat_list = set(penguat_list)

In [7]:
with open('filtered-dumping-wiki.txt') as fopen:
    data = list(filter(None, fopen.read().split('\n')))
    
data = [i for i in data if len(i) >= 2]

len(data)

2037249

In [8]:
tokenizer = malaya.preprocessing.TOKENIZER().tokenize

In [9]:
import copy
import re
from malaya.text.regex import _expressions

def reset_t(tokens):
    t = []
    for i in range(len(tokens)):
        t.append([tokens[i], 2])
    return t

# [penguat] [kata sifat] -> [penguat] [kata sifat] [end_penguat_list]
# paling tinggi -> paling tinggi sangat
def augment_5_0(t, row):
    text, tokens, tokens_lower, penguat = row
    for word in penguat:
        try:
            i = tokens_lower.index(word) + 1
            if tokens[i][0].isupper():
                continue
            ends = random.choice(end_penguat_list)
            t[i][0] = f'{tokens[i]} {ends}'
            t[i][1] = 5
        except Exception as e:
            print(e)
            pass

In [11]:
results = []
for text in tqdm(['gunung itu sangat tinggi']):
    tokens = tokenizer(text)
    t = reset_t(tokens)
    t_ = copy.deepcopy(t)
    tokens_lower = tokenizer(text.lower())
    set_tokens = set(tokens_lower)
    r_penguat_list = set_tokens & set_penguat_list
    r = (t, tokens, tokens_lower, r_penguat_list)
    augment_5_0(t_, r)
    results.append((t, t_))

100%|██████████| 1/1 [00:00<00:00, 2981.03it/s]


In [12]:
results

[([['gunung', 2], ['itu', 2], ['sangat', 2], ['tinggi', 2]],
  [['gunung', 2], ['itu', 2], ['sangat', 2], ['tinggi sungguh', 5]])]